# Testing Different Seasons and Leagues Used to Train an EPL Model 

When searching for data, it’s essential to understand how a model's performance changes with different training datasets. This helps us determine what data to prioritize for optimal results.

In this notebook, we examine how the choice of leagues in the training data affects model performance when evaluated exclusively on the English Premier League (EPL). To do this, we will:

0. [Establish a Baseline Model](#baseline-training-on-all-available-epl-data-only):
    - The model will predict second-half-of-season results using first-half season statistics as features.
    - It will be trained on all available EPL data*.

1. [Experiment with Data Cutoffs](#experiment-1-cutting-training-data-to-recent-seasons):
    - We will assess the impact of restricting the training data to more recent EPL seasons by removing older seasons.

2. [Incorporate Additional Leagues](#experiment-2-extending-training-data-to-other-top-leagues-all-seasons):
    - We will explore how training on multiple leagues affects model performance when applied to EPL predictions.

3. [Combine Both Strategies](#experiment-3-training-on-all-top-leagues-recent-seasons):
    - We will train the model using data from multiple leagues but only from recent seasons. This will help determine whether including more leagues can compensate for a lack of historical EPL data should we encounter this issue in our search.

4. [Remove EPL Data Completely](#experiment-4-training-on-non-epl-data-only):
    - Finally, as a bonus, we will train on non-EPL data and test on EPL data to understand how important it is to represent the leagues we are predicting for in our training set.

*The dataset we are using backdates to 2010 however there are additional seasons available from the source.

By conducting these experiments, we aim to identify the most effective training data composition for predicting EPL outcomes.

The modelling we use in this notebook (particularly the concept of using aggregated first-half-of-season stats to predict results in the second half) does not necessarily reflect how our future model(s) will work. This is meerly a method that sufficiently represents the likely complexity of our future model(s) whilst allowing us to use easily available back-dated data.

The data used in this notebook was sourced from [Football-Data.co.uk](https://Football-Data.co.uk).



## Experimentation Prep

In [1]:
import pandas as pd
import mlflow
from functools import partial

import sys
import os

sys.path.append(os.path.abspath(os.path.join("../src")))
from src.ingestion.preprocess import create_dataframe
from experiment import run_experiment
from helper_functions import generate_random_string

In [2]:
raw_data = pd.read_csv("../data/raw_games.csv")

In [3]:
data = create_dataframe(raw_data)

2025-02-22 09:12:18.533 | INFO     | legacy.preprocess:create_dataframe:13 - Starting dataset creation
2025-02-22 09:12:18.538 | INFO     | legacy.preprocess:create_dataframe:113 - Home and away dataframes created succesfully
2025-02-22 09:12:18.543 | INFO     | legacy.preprocess:create_dataframe:119 - Home and away dataframes succesfully combined and grouped
2025-02-22 09:12:18.554 | INFO     | legacy.preprocess:create_dataframe:151 - Team stats succesfully merged onto game data
2025-02-22 09:12:18.556 | INFO     | legacy.preprocess:create_dataframe:156 - Duplicate columns successfully dropped
2025-02-22 09:12:18.561 | INFO     | legacy.preprocess:create_dataframe:170 - Dataset creation complete
2025-02-22 09:12:18.561 | INFO     | legacy.preprocess:create_dataframe:171 - Dataset length: 16981


In [4]:
def list_remove(lst: list, remove: list) -> list:
    return [x for x in lst if x not in remove]

In [5]:
non_features = [
    "season",
    "div",
    "date",
    "h_team",
    "a_team",
    "ftr",
    "b365h",
    "b365d",
    "b365a",
    "h_win",
    "bookies_prob",
]
FEATURE_NAMES = list_remove(data.columns, non_features)
ALL_SEASONS = list(data["season"].drop_duplicates())
TEST_SEASONS = ["23_24", "24_25"]
VAL_SEASONS = ["21_22", "22_23"]

In [6]:
columns = FEATURE_NAMES + ["bookies_prob", "div", "season", "h_win"]

data = data[columns]

train_full = data[~data["season"].isin(VAL_SEASONS + TEST_SEASONS)]
train_full = train_full.drop(columns=["bookies_prob"])

val = data[data["season"].isin(VAL_SEASONS) & (data["div"] == "E0")]
val = val.drop(columns=["div"])

In [7]:
seasons_13 = list_remove(ALL_SEASONS, ["10_11", "11_12", "12_13"])
seasons_17 = list_remove(seasons_13, ["13_14", "14_15", "15_16"])

In [8]:
training_experiment = partial(
    run_experiment,
    experiment_name="data-needs-experiment",
    val_data=val,
    hidden_units=None,
    learning_rate=0.001,
    num_epochs=10000,
    num_samples=1000,
    num_batches=1,
    league_tag="EPL",
    return_model=False,
)

In [9]:
def get_run_metrics(run_id: str) -> pd.DataFrame:
    metrics = mlflow.search_runs(filter_string=f"tags.run_id = '{run_id}'")[
        [
            "tags.run_description",
            "params.n_train",
            "params.num_train_seasons",
            "metrics.train_auc",
            "metrics.val_auc",
            "metrics.train_mse",
            "metrics.val_mse",
            "metrics.val_mse_diff",
            "metrics.val_auc_diff",
        ]
    ]

    return metrics

## Baseline: Training on All Available EPL Data Only

In [10]:
train_bl = train_full.copy()
train_bl = train_bl.loc[train_bl["div"] == "E0"]
train_bl = train_bl.drop(columns="div")

In [11]:
bl_run_id = generate_random_string()
print(f"Experiment run ID: {bl_run_id}")

training_experiment(
    run_id=bl_run_id,
    train_data=train_bl,
    run_description="training on 2010-2021 EPL data",
)

Experiment run ID: AZmwvb4r


Validation Output Sampling Progress: 100%|██████████| 1000/1000 [00:25<00:00, 39.43it/s]


🏃 View run unruly-frog-994 at: http://localhost:5001/#/experiments/3/runs/e37467b8e994464da1b2aa8310294b35
🧪 View experiment at: http://localhost:5001/#/experiments/3


In [12]:
run_data_bl = get_run_metrics(bl_run_id)
run_data_bl

,tags.run_description,params.n_train,params.num_train_seasons,metrics.train_auc,metrics.val_auc,metrics.train_mse,metrics.val_mse,metrics.val_mse_diff,metrics.val_auc_diff
0,training on 2010-2021 EPL data,3194,17,0.712732,0.72021,0.215064,0.210865,0.004528,-0.022212


## Experiment 1: Cutting Training Data To Recent Seasons 

### Part A: 2013-2021

In [13]:
train_e1a = train_full.copy()
train_e1a = train_e1a.loc[
    (train_e1a["div"] == "E0") & (train_e1a["season"].isin(seasons_13))
]
train_e1a = train_e1a.drop(columns="div")

In [14]:
e1a_run_id = generate_random_string()
print(f"Experiment run ID: {e1a_run_id}")

training_experiment(
    run_id=e1a_run_id,
    train_data=train_e1a,
    run_description="training on 2013-2021 EPL data",
)

Experiment run ID: RW3BrsWd


Validation Output Sampling Progress: 100%|██████████| 1000/1000 [00:25<00:00, 38.85it/s]


🏃 View run abrasive-yak-191 at: http://localhost:5001/#/experiments/3/runs/fc0cc3b6dc6549d2bf228a69755a3bad
🧪 View experiment at: http://localhost:5001/#/experiments/3


In [15]:
run_data_e1a = get_run_metrics(e1a_run_id)
run_data_all = pd.concat([run_data_bl, run_data_e1a])
run_data_e1a

,tags.run_description,params.n_train,params.num_train_seasons,metrics.train_auc,metrics.val_auc,metrics.train_mse,metrics.val_mse,metrics.val_mse_diff,metrics.val_auc_diff
0,training on 2013-2021 EPL data,2632,14,0.714558,0.71522,0.214441,0.213098,0.006761,-0.027202


### Part B: 2017-2021

In [16]:
train_e1b = train_full.copy()
train_e1b = train_e1b.loc[
    (train_e1b["div"] == "E0") & (train_e1b["season"].isin(seasons_17))
]
train_e1b = train_e1b.drop(columns="div")

In [17]:
e1b_run_id = generate_random_string()
print(f"Experiment run ID: {e1b_run_id}")

training_experiment(
    run_id=e1b_run_id,
    train_data=train_e1b,
    run_description="training on 2017-2021 EPL data",
)

Experiment run ID: INOQnuhA


Validation Output Sampling Progress: 100%|██████████| 1000/1000 [00:25<00:00, 38.80it/s]


🏃 View run incongruous-bug-768 at: http://localhost:5001/#/experiments/3/runs/6b609b82e0f946afa850b012fe954d8f
🧪 View experiment at: http://localhost:5001/#/experiments/3


In [18]:
run_data_e1b = get_run_metrics(e1b_run_id)
run_data_all = pd.concat([run_data_all, run_data_e1b])
run_data_e1b

,tags.run_description,params.n_train,params.num_train_seasons,metrics.train_auc,metrics.val_auc,metrics.train_mse,metrics.val_mse,metrics.val_mse_diff,metrics.val_auc_diff
0,training on 2017-2021 EPL data,2072,11,0.720055,0.710689,0.212826,0.214242,0.007906,-0.031733


## Experiment 2: Extending Training Data To Other Top Leagues (All Seasons)

In [19]:
train_e2 = train_full.copy()
train_e2 = train_e2.drop(columns="div")

In [20]:
e2_run_id = generate_random_string()
print(f"Experiment run ID: {e2_run_id}")

training_experiment(
    run_id=e2_run_id,
    train_data=train_e2,
    run_description="training on 2010-2021 all leagues data",
)

Experiment run ID: HzIW69rq


Validation Output Sampling Progress: 100%|██████████| 1000/1000 [00:26<00:00, 37.37it/s]


🏃 View run abrasive-goose-478 at: http://localhost:5001/#/experiments/3/runs/6a6aa4fd26324136be70336d5443a4c3
🧪 View experiment at: http://localhost:5001/#/experiments/3


In [21]:
run_data_e2 = get_run_metrics(e2_run_id)
run_data_all = pd.concat([run_data_all, run_data_e2])
run_data_e2

,tags.run_description,params.n_train,params.num_train_seasons,metrics.train_auc,metrics.val_auc,metrics.train_mse,metrics.val_mse,metrics.val_mse_diff,metrics.val_auc_diff
0,training on 2010-2021 all leagues data,14038,17,0.689589,0.715335,0.220903,0.212167,0.00583,-0.027087


## Experiment 3: Training on All Top Leagues (Recent Seasons)

### Part A: 2013-2021

In [22]:
train_e3a = train_full.copy()
train_e3a = train_e3a.loc[train_e3a["season"].isin(seasons_13)]
train_e3a = train_e3a.drop(columns="div")

In [23]:
e3a_run_id = generate_random_string()
print(f"Experiment run ID: {e3a_run_id}")

training_experiment(
    run_id=e3a_run_id,
    train_data=train_e3a,
    run_description="training on 2013-2021 all leagues data",
)

Experiment run ID: pDlrE0U1


Validation Output Sampling Progress: 100%|██████████| 1000/1000 [00:27<00:00, 36.51it/s]


🏃 View run enchanting-hog-491 at: http://localhost:5001/#/experiments/3/runs/68eef320305e4d59bd5d33660dc39de3
🧪 View experiment at: http://localhost:5001/#/experiments/3


In [24]:
run_data_e3a = get_run_metrics(e3a_run_id)
run_data_all = pd.concat([run_data_all, run_data_e3a])
run_data_e3a

,tags.run_description,params.n_train,params.num_train_seasons,metrics.train_auc,metrics.val_auc,metrics.train_mse,metrics.val_mse,metrics.val_mse_diff,metrics.val_auc_diff
0,training on 2013-2021 all leagues data,11325,14,0.692336,0.715363,0.22009,0.211851,0.005515,-0.027058


### Part B: 2017-2021

In [25]:
train_e3b = train_full.copy()
train_e3b = train_e3b.loc[train_e3b["season"].isin(seasons_17)]
train_e3b = train_e3b.drop(columns="div")

In [26]:
e3b_run_id = generate_random_string()
print(f"Experiment run ID: {e3b_run_id}")

training_experiment(
    run_id=e3b_run_id,
    train_data=train_e3b,
    run_description="training on 2017-2021 all leagues data",
)

Experiment run ID: tqMObIVT


Validation Output Sampling Progress: 100%|██████████| 1000/1000 [00:27<00:00, 35.83it/s]


🏃 View run upbeat-auk-30 at: http://localhost:5001/#/experiments/3/runs/1691c740662a4ca1998dc370e12b7e61
🧪 View experiment at: http://localhost:5001/#/experiments/3


In [27]:
run_data_e3b = get_run_metrics(e3b_run_id)
run_data_all = pd.concat([run_data_all, run_data_e3b])
run_data_e3b

,tags.run_description,params.n_train,params.num_train_seasons,metrics.train_auc,metrics.val_auc,metrics.train_mse,metrics.val_mse,metrics.val_mse_diff,metrics.val_auc_diff
0,training on 2017-2021 all leagues data,8640,11,0.693169,0.714675,0.219878,0.212451,0.006114,-0.027747


## Experiment 4: Training on Non-EPL Data Only

In [28]:
train_e4 = train_full.copy()
train_e4 = train_e4.loc[train_e4["div"] != "E0"]
train_e4 = train_e4.drop(columns="div")

In [29]:
e4_run_id = generate_random_string()
print(f"Experiment run ID: {e4_run_id}")

training_experiment(
    run_id=e4_run_id,
    train_data=train_e4,
    run_description="training on 2010-2021 EPL removed data",
)

Experiment run ID: 0XJDjvgB


Validation Output Sampling Progress: 100%|██████████| 1000/1000 [00:26<00:00, 37.38it/s]


🏃 View run mercurial-bird-932 at: http://localhost:5001/#/experiments/3/runs/8bca2e71a17444b1980ae9265bbc58d0
🧪 View experiment at: http://localhost:5001/#/experiments/3


In [30]:
run_data_e4 = get_run_metrics(e4_run_id)
run_data_all = pd.concat([run_data_all, run_data_e4])
run_data_e4

,tags.run_description,params.n_train,params.num_train_seasons,metrics.train_auc,metrics.val_auc,metrics.train_mse,metrics.val_mse,metrics.val_mse_diff,metrics.val_auc_diff
0,training on 2010-2021 EPL removed data,10844,16,0.683575,0.711205,0.222049,0.213083,0.006746,-0.031217


## Comparison and Conclusion

In [31]:
run_data_all.drop(
    columns=["metrics.train_auc", "metrics.train_mse"]
).sort_values("metrics.val_mse", ascending=True)

,tags.run_description,params.n_train,params.num_train_seasons,metrics.val_auc,metrics.val_mse,metrics.val_mse_diff,metrics.val_auc_diff
0,training on 2010-2021 EPL data,3194,17,0.720210,0.210865,0.004528,-0.022212
0,training on 2013-2021 all leagues data,11325,14,0.715363,0.211851,0.005515,-0.027058
0,training on 2010-2021 all leagues data,14038,17,0.715335,0.212167,0.005830,-0.027087
0,training on 2017-2021 all leagues data,8640,11,0.714675,0.212451,0.006114,-0.027747
0,training on 2010-2021 EPL removed data,10844,16,0.711205,0.213083,0.006746,-0.031217
0,training on 2013-2021 EPL data,2632,14,0.715220,0.213098,0.006761,-0.027202
0,training on 2017-2021 EPL data,2072,11,0.710689,0.214242,0.007906,-0.031733


A few important points:

1. We are dealing with very small discrepancies in the evaluation metrics that, at face value, seem negligible. However, when we examine the `val_mse_diff` and `val_auc_diff` columns—which represent the disparity in MSE and AUC between our model and the bookies' odds—we see that these differences are not as insignificant as they first appear. For example, while a $0.001$ difference in MSE might seem inconsequential, if the total difference between our model’s MSE and that of the bookies' odds is only $0.02$, then suddenly that $0.001$ decrease becomes more meaningful and something we should strive to avoid. It is important therefore that we use the `_diff` columns for context when disucssing changes in metrics between runs.

2. Due to the model's feature selection, we are only utilising half of the available data points per season (i.e., games in the second half of the season). This means that any conclusions we attribute to insufficient data may not hold to the same extent (if at all) if we adopt an approach that allows us to use most or all of each season’s games.

3. Similarly, we cannot rule out that a different approach could lead to entirely different conclusions for other reasons. For example, if we were to find different features that (for whatever reason) carry more predictive signal in later seasons, a model that heavily relies on such features would likely benefit from a more recent training set. However, the goal of this notebook is to develop a rough understanding of how substituting earlier EPL seasons for more recent foreign seasons might affect model performance. This is a trade-off we may have to consider if we want a more comprehensive feature set.